# EasyVitessce Example: Select a gene interactively, then use the value

## Import EasyVitessce

This import statement enables interactive plots by default.
Refer to the [EasyVitessce documentation](https://vitessce.github.io/easy_vitessce/) for how to disable interactive plotting or configure other behaviors.

In [ ]:
import easy_vitessce as ev

## Utility dependencies

First, we import utility dependencies which will be used to download the example dataset and manipulate file paths, zip files, and JSON files.

In [ ]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
import zipfile
import json

## Dependencies for data structures

In [ ]:
from spatialdata import read_zarr

## Download example dataset

In [ ]:
data_dir = "data"
zip_filepath = join(data_dir, "visium.spatialdata.zarr.zip")
spatialdata_filepath = join(data_dir, "visium.spatialdata.zarr")

The following code uses Python's `urlretrieve` to download the SpatialData object as a zip file, then unzips the file using the `zipfile` module.

In [ ]:
if not isdir(spatialdata_filepath):
    if not isfile(zip_filepath):
        os.makedirs(data_dir, exist_ok=True)
        urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/visium.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(data_dir)
        os.rename(join(data_dir, "data.zarr"), spatialdata_filepath)

## Read the example SpatialData object

In [ ]:
sdata = read_zarr(spatialdata_filepath)
sdata

## Plot the data

Store the return value of `.pl.show()` in a variable.
See more details at https://vitessce.github.io/easy_vitessce/advanced.html#access-the-vitessce-configuration

In [ ]:
vw = sdata.pl.render_images("ST8059050_hires_image").pl.render_shapes("ST8059050", color="Fth1").pl.show("ST8059050")
vw

## Get the selected gene

See more details at https://vitessce.github.io/easy_vitessce/advanced.html#access-values-from-the-coordination-space

In [ ]:
current_config = vw._config
coordination_type = "featureSelection"
coordination_object = current_config["coordinationSpace"][coordination_type]
coordination_object

In [ ]:
selected_gene_ids = coordination_object['A']
selected_gene_ids

The value of `featureSelection[coordinationScope]` will be an array (to allow for multi-selection, which is not used here but is important for other plot types such as the dot plot).

In [ ]:
first_selected_gene_id = selected_gene_ids[0]
first_selected_gene_id

## Print the selected gene to the Jupyter log console upon change

In the code above, the value of the variable `current_config` will be populated upon execution of the notebook cell.
However, if the user selects a different gene within the Vitessce widget AFTER running the line  `current_config = vw._config`, the config value (and its downstream derived values such as `first_selected_gene_id`) will become stale.

To resolve this, we can provide a callback function which will be executed on any change to `vw._config`. (This is possible because VitessceWidget inherits from AnyWidget which inherits the `.observe` [method from ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#traitlet-events)).

Note: to see the printed values, you may need to open the Jupyer Log Console (View -> Show Log Console in JupyterLab).

In [ ]:
def on_config_change(change):
    # Diff change.old and change.new.
    prev_selected_gene_ids = change.old["coordinationSpace"]["featureSelection"]["A"]
    next_selected_gene_ids = change.new["coordinationSpace"]["featureSelection"]["A"]
    if len(next_selected_gene_ids) == 1 and prev_selected_gene_ids != next_selected_gene_ids:
        # Print the new value if it was different from the previous value.
        print(next_selected_gene_ids[0])

vw.observe(on_config_change, names=['_config'])